In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
import seaborn as sns
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [3]:
metrics = os.listdir('./raw')
metrics = [m for m in metrics if 'csv' in m]

In [33]:
%%script false
def sortByIndex(metric):
    data = pd.read_csv('./raw/%s' % metric)
    miss1 = pd.read_csv('./raw/missed/%s' % metric)
    miss2 = pd.read_csv('./raw/missed1/%s' % metric)
    data = pd.concat([data, miss1, miss2], axis=0).sort_values(by=['index'])
    data.to_csv('./raw/%s' % metric, index=None)
    data.to_csv('./processed/%s' % metric, index=None)
for m in metrics:
    sortByIndex(m)

Couldn't find program: 'false'


In [7]:
# %%script false
latency = pd.read_csv('./processed/latency.csv')
missed_index = list(set(range(5000)) - set(latency['index']))
full_schedule = pd.read_csv('schedule.csv')
missed_schedule = full_schedule.iloc[missed_index]
missed_schedule.to_csv('missed_schedule.csv', index=None)

In [35]:
def plot_data(X, col):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X.index, y=X[col], mode='markers', name=col))
    fig.update_layout(height=400, width=800, showlegend=True)
    fig.show()

for m in metrics:
    print(m)
    data = pd.read_csv('./processed/%s' % m, index_col=0)

    # drop constant columns
    data = data.loc[:, (data != data.iloc[0]).any()] 
    data_cols = data.columns

    # scaling data
    scaler = MinMaxScaler()
    data[data_cols] = scaler.fit_transform(data[data_cols])
    data.to_csv('./processed/%s' % m)

latency.csv
states_25th.csv
states_50th.csv
states_75th.csv
states_90th.csv
states_95th.csv
states_99th.csv
states_max.csv
states_mean.csv
states_median.csv
states_min.csv
states_std.csv


In [14]:
throughput_metrics = pd.DataFrame([])
for m in metrics:
    data = pd.read_csv('./processed/%s' % m)
    if 'server_broker_topics_AllTopicsBytesOut' in data.columns:
        throughput_metrics[m.split('.')[0].split('_')[1]] = data['server_broker_topics_AllTopicsBytesOut']
throughput_metrics.index = data.index
throughput_metrics.index.name = 'index'
throughput_metrics.to_csv('./processed/throughput.csv')

In [8]:
schedule = pd.read_csv('schedule.csv')
compression_type = ['none', 'gzip', 'snappy']
ct = schedule['producer->compression_type']
for i in range(len(ct)):
    ct[i] = compression_type.index(ct[i])
schedule['producer->compression_type'] = ct
schedule.to_csv('schedule1.csv', index=None)

<ipython-input-8-56c7d732023d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ct[i] = compression_type.index(ct[i])
